In [1]:
#init network functoin
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [2]:
from random import seed
from random import random

In [38]:
seed(1)
network = initialize_network(2,1,2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


1 hidden layer, with two weights plus bias
ouput layer with two neurons each with 1 wiehgt plus a bias

In [8]:
[random() for i in range(3)]

[0.7887233511355132, 0.0938595867742349, 0.02834747652200631]

## Forward propogate

carry input through the network

need neuron activation
nueron transfer
and forward prop

In [24]:
#calculate neruon activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i]*inputs[i]
    return(activation)

#transfer function
def transfer(activation):
    return(1.0 / (1.0 + exp(-activation)))

we work through each layer of our network calculatin the outputs for each neruone. all the outputs from one layer become inputs to the neurons on the next layer

In [34]:
def forward_propogate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'],inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return(inputs)

In [62]:
from math import exp
# test forward propagation
network = initialize_network(2,1,2)
row = [1, 0, None]
output = forward_propogate(network, row)
print(output)

[0.648629545757926, 0.7969578641752217]


wooohoo

## Back propagate error

way to train the weights

eror is calculated between the expected outputs and the outpts forward propagate from the network. These errors are then propogate backward through the netowrk form the output layer to the hidden layer, assignling blacem for the error

In [55]:
#transfer derivative
def transfer_derivative(output):
    return(output * (1.0 - output))

Error propogation:

error = (exepcted - output)*transferddeivatie(output)

which is just chain rule wrt to the ouput

the error signal for a neuron in the hidden layer is calculated as the weighted error of each neurone in the output layer. think of the error traveling back along the weights of the ouput layer to the neurons in the hidden layer

error = (weight_k * error_j) * transfer_derivative(output)

where error_j is the error signal from the jth neurone in the output layer, weight_k is the weight that connects the kth neuorn to the current neuron and output if the outpur for the current neueron


In [56]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [63]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)
 
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
                neuron = layer[j]
                neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
 
#test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
backward_propagate_error(network, expected)
for layer in network:
	print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}]


In [72]:
for i in network[0+1]:
    print(i['weights'])
    print(i['delta'])

[0.2550690257394217, 0.49543508709194095]
-0.14619064683582808
[0.4494910647887381, 0.651592972722763]
0.0771723774346327


## Train the network

the network is trained using stoachsit gradient decent

this involves using mulitple iterations of exposing a training dataset to the network and for each row of data forward propgating the inputs, backpropgating the error graidents and updating the weights

defined as:
update wieghts
train network

weight = weight + learning_rate * error * input

notes on learning rates, it controls how much to change the weight vector. for example, a small value of 0.1 will update 10% of the amount of time it possibly could be updated.

small learning rates are preferred that cause slower learning over a larger number of training iterations. This increase the likelihood of the network finding a good set of weight across all layers rather than the fastest set of weights that minimuze the error

In [73]:
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']
                